# Imports

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import string
import re
from datetime import datetime

# Import Data

In [2]:
process = pd.read_csv('https://github.com/ftmnl/asr/raw/main/data/allExport.csv', sep='|')

In [3]:
process

,file_name_sort,content
0,.DS_Store,\nSize:6148\nDate:2021-04-21\nTime:09:29:32.96...
1,.DS_Store,\nSize:6148\nDate:2021-04-21\nTime:09:41:28.04...
2,.DS_Store,\nSize:6148\nDate:2021-04-21\nTime:09:43:31.22...
3,.DS_Store,\nSize:6148\nDate:2021-04-21\nTime:09:45:45.48...
4,.DS_Store,\nSize:6148\nDate:2021-04-21\nTime:09:52:47.55...
...,...,...
2543,NaN,NaN
2544,NaN,NaN
2545,NaN,NaN
2546,NaN,NaN


In [8]:
#remove noise
process = process.dropna()

process = process.rename(columns = {"file_name_sort": "title", "content": "abstract"})

remove = ['.DS_Store', 'NaN', 'Readme.md']
process = process[~process.title.isin(remove)]

# Transform Content

In [9]:
#abstract html to string

translate_table = dict((ord(char), None) for char in string.punctuation)

def prettify(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = text.replace("\r", "")
    #text = text.replace('\n', '')
    #text = text.translate(translate_table)
    return str(text)

process.abstract = process.abstract.apply(prettify)

# Search formats

In [10]:
#extract id, type and date from title

import re

numberlist = []
typelist = []
datelist = []

for title in process.title:
    id = re.search('^[0-9\.]+', title)
    if id == None:
        numberlist.append(None)
    else: 
        numberlist.append(id.group(0))

    type = re.search('(?<=[0-9\_]\_)[a-z A-Z]+(?=\_)', title)
    if type != None:
        typelist.append(type.group(0))
    else:
        typelist.append("Onbekend")
        
    date = re.search("[0-9-?]+(?=.pdf$)", title)
    if date == None:
        datelist.append(None)
    else: 
        try: datelist.append(datetime.strptime(date.group(0), '%d-%m-%Y'))
        except: 
            try: datelist.append(datetime.strptime(date.group(0), '%-d-%-m-%Y'))
            except: datelist.append(date.group(0))

process["id"] = numberlist
process["type"] = typelist
process["date"] = datelist


In [11]:
#Improve some Dating

betterDate = []

for index, row in process.iterrows():
    regexDate = re.search("(?<=Date : )[0-9-]{3,5}-20[0-9]{2} [0-9:]{8}", row.abstract)
    if regexDate != None:
        date = datetime.strptime(regexDate.group(0), '%d-%m-%Y %H:%M:%S')
        betterDate.append(date)
    else: 
        betterDate.append(None)

process["betterDate"] = betterDate

# Clean Data

In [12]:
def cleanTitle(title):
    title = re.sub('^[0-9\.]+_+[a-z A-Z]+_', '', title)
    title = re.sub('[0-9\-]+.pdf$', '', title)
    title = re.sub('.msg_', ' ', title)
    return title

process.title = process.title.apply(cleanTitle)

# Export processing

In [13]:
process[['id','type','date','betterDate','title','abstract']].to_excel('../pre-processing/preprocessed.xlsx')